# 2$\nu_x - \nu_y$ Resonance

0. [Functions](#chapter0)
1. [Pulse/AC Skew Sextupole](#chapter1)\
    1.1 [Single Particle](#chapter1.1)\
    1.2 [Multiparticle](#chapter1.2)
2. [Resonance Crossing](#chapter2)\
    2.1 [Single Particle](#chapter2.1)\
    2.2 [Multiparticle](#chapter2.2)
3. [AC Skew Sextupole](#chapter3)\
    3.1 [Single Particle](#chapter3.1)\
    3.2 [Multiparticle](#chapter3.2)  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import matplotlib.animation as animation
from IPython.display import display, Math
import PyQt5
%matplotlib qt

#Run all functions before running anything else

# 0. Functions <a name="chapter0"></a>

In [2]:
def epsilon_single(a, b, g, x, xp): return (g*x**2 + 2*a*x*xp + b*xp**2)/2

# http://cds.cern.ch/record/242313/files/p89.pdf
def epsilon_multi(X, Xp): 
    r = np.mean(X*Xp)/np.sqrt(np.mean(X**2)*np.mean(Xp**2))
    return 4*np.std(X)*np.std(Xp)*np.sqrt(1 - r**2)

def M_phase(Twiss):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y = Twiss
    Sx, Sy = np.sin(2*np.pi*nu_x), np.sin(2*np.pi*nu_y)
    Cx, Cy = np.cos(2*np.pi*nu_x), np.cos(2*np.pi*nu_y)
    M = np.array([np.array([Cx + ax*Sx, bx*Sx, 0, 0]),
                 np.array([-gx*Sx, Cx - ax*Sx, 0, 0]),
                 np.array([0, 0, Cy + ay*Sy, by*Sy]),
                 np.array([0, 0, -gy*Sy, Cy - ay*Sy])])
    return M

In [14]:
def Animation_single(X_hist, buffer, rate):
    fig, ax = plt.subplots(figsize = (10, 10))
    xPulseSingle, = ax.plot(X_hist[0][0], X_hist[1][0], 'o', label = 'x')
    yPulseSingle, = ax.plot(X_hist[2][0], X_hist[3][0], 'o', label = 'y')
    xbound = max(np.max(X_hist[0]), np.max(X_hist[2])) + .001
    ybound = max(np.max(X_hist[1]), np.max(X_hist[3])) + .001
    ax.set_xlim([-xbound, xbound])
    ax.set_ylim([-ybound, ybound])
    ax.set_xlabel("z")
    ax.set_ylabel(r"$z^'$")
    ax.legend()
    title = ax.text(0.5, .95, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                    transform=ax.transAxes, ha="center")

    def animate(i):
        if i < buffer:
            xPulseSingle.set_xdata(X_hist[0][0: i])
            xPulseSingle.set_ydata(X_hist[1][0: i])
            yPulseSingle.set_xdata(X_hist[2][0: i])
            yPulseSingle.set_ydata(X_hist[3][0: i])
        else:
            xPulseSingle.set_xdata(X_hist[0][i - buffer: i])
            xPulseSingle.set_ydata(X_hist[1][i - buffer: i])
            yPulseSingle.set_xdata(X_hist[2][i - buffer: i])
            yPulseSingle.set_ydata(X_hist[3][i - buffer: i])
        title.set_text("Phase Space at turn %d"%i)
        return xPulseSingle, yPulseSingle, title,

    def init():
        xPulseSingle.set_ydata(np.ma.array(X_hist[0], mask=True))
        yPulseSingle.set_ydata(np.ma.array(X_hist[2], mask=True))
        return xPulseSingle, yPulseSingle,

    N_turns = len(X_hist[0])
    ani = animation.FuncAnimation(fig, animate, np.arange(1, N_turns), init_func=init,
        interval=rate, blit=True)
    
    return ani


In [4]:
# Visualize exchange
def Exchange_single(X_hist, save_txt, ncol = 5, size = 1):
    nrow = 2
    fig, Ax = plt.subplots(nrow, ncol, figsize = (25, 8))
    fig.suptitle('Phase Space For One Particle')
    N_turns = len(X_hist[0])
    xbound = max(np.max(X_hist[0]), np.max(X_hist[2])) + .001
    ybound = max(np.max(X_hist[1]), np.max(X_hist[3])) + .0005

    for i in range(nrow):
        xlab = ""
        ylab = ""
        if i == 0:
            xlab = "x"
            Ax[i][0].set_ylabel(r"$x^'$")
            
        if i == 1:
            xlab = "y"
            Ax[i][0].set_ylabel(r"$y^'$")

        for j in range(ncol):
            initial = N_turns*j//ncol
            final = N_turns*(j + 1)//ncol

            if final >= N_turns:
                final = -1
            Ax[i][j].plot(X_hist[i*2][initial: final], 
                          X_hist[i*2 + 1][initial: final], "o", mfc = 'black', ms = size)
            Ax[i][j].set_xlabel(xlab)

            Ax[i][j].set_xlim([-xbound, xbound])
            Ax[i][j].set_ylim([-ybound, ybound])
            if i == 0:
                if final == -1:
                    final = N_turns
                Ax[0][j].set_title("%d - %d turns"%(initial, final))
            if j != 0:
                Ax[i][j].get_yaxis().set_visible(False)
    fig.savefig(save_txt)
    return Ax       

In [5]:
def Animation_multi(X_hist, buffer, rate, nrow = 2, ncol = 5, n = 0):
    fig, ax = plt.subplots(figsize = (15, 10))

    x_hist = X_hist[0].T

    xPulseSingle, = ax.plot(x_hist[0], x_hist[1], 'o', label = 'x')
    yPulseSingle, = ax.plot(x_hist[2], x_hist[3], 'o', label = 'y')
    xPulseSingle0, = ax.plot(x_hist[0][n], x_hist[1][n], 'o', mec = 'black', mew = '1', mfc = 'blue', ms = 10, label = 'x0')
    yPulseSingle0, = ax.plot(x_hist[2][n], x_hist[3][n], 'o', mec = 'black', mew = '1', mfc = 'red', ms = 10, label = 'y0')
    xbound = []
    ybound = []
    factor = N_turns//(ncol - 1)
    for i in range(nrow):
        for j in range(ncol):
            index = factor*j
            if factor*j == N_turns:
                index = -1
            X_hist_sub = X_hist[index].T
            xbound.append(np.max(np.abs(X_hist_sub[i*2])))
            ybound.append(np.max(np.abs(X_hist_sub[i*2 + 1])))

    xbound_Max = np.max(xbound) + .001
    ybound_Max = np.max(ybound) + .0005
    ax.set_xlim([-xbound_Max, xbound_Max])
    ax.set_ylim([-ybound_Max, ybound_Max])
    ax.set_xlabel("z")
    ax.set_ylabel(r"$z^'$")
    ax.legend()
    title = ax.text(0.5, .9, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                    transform=ax.transAxes, ha="center")

    def animate(i):
        xPulseSingle.set_xdata(X_hist[i].T[0])
        xPulseSingle.set_ydata(X_hist[i].T[1])
        yPulseSingle.set_xdata(X_hist[i].T[2])
        yPulseSingle.set_ydata(X_hist[i].T[3])
        if i < buffer:
            xPulseSingle0.set_xdata(X_hist[0: i].T[0][n])
            xPulseSingle0.set_ydata(X_hist[0: i].T[1][n])
            yPulseSingle0.set_xdata(X_hist[0: i].T[2][n])
            yPulseSingle0.set_ydata(X_hist[0: i].T[3][n])
        else:
            xPulseSingle0.set_xdata(X_hist[i - buffer: i].T[0][n])
            xPulseSingle0.set_ydata(X_hist[i - buffer: i].T[1][n])
            yPulseSingle0.set_xdata(X_hist[i - buffer: i].T[2][n])
            yPulseSingle0.set_ydata(X_hist[i - buffer: i].T[3][n])
        title.set_text("Phase Space at turn %d"%i)
        return xPulseSingle, xPulseSingle0, yPulseSingle, yPulseSingle0, title,

    def init():
        xPulseSingle.set_ydata(np.ma.array(x_hist[0], mask=True))
        yPulseSingle.set_ydata(np.ma.array(x_hist[2], mask=True))
        xPulseSingle0.set_ydata(np.ma.array(x_hist[0], mask=True))
        yPulseSingle0.set_ydata(np.ma.array(x_hist[2], mask=True))
        return xPulseSingle, xPulseSingle0, yPulseSingle, yPulseSingle0,

    ani = animation.FuncAnimation(fig, animate, np.arange(1, N_turns), init_func=init,
        interval=rate, blit=True)

    return ani

In [6]:
# Visualize exchange
def Exchange_multi(X_hist, save_txt, ncol = 5, size = 1):
    nrow = 2

    N_turns = len(X_hist)
    N_part = len(X_hist[0])
    fig, Ax = plt.subplots(nrow, ncol, figsize = (25, 8))
    fig.suptitle('Phase Space For %d Particles'%N_part)
    xbound = []
    ybound = []
    factor = N_turns//(ncol - 1)
    for i in range(nrow):
        for j in range(ncol):
            index = factor*j
            if factor*j == N_turns:
                index = -1
            X_hist_sub = X_hist[index].T
            xbound.append(np.max(np.abs(X_hist_sub[i*2])))
            ybound.append(np.max(np.abs(X_hist_sub[i*2 + 1])))

    xbound_Max = np.max(xbound) + .001
    ybound_Max = np.max(ybound) + .0005

    for i in range(nrow):
        xlab = ""
        ylab = ""
        if i == 0:
            xlab = "x"
            Ax[i][0].set_ylabel(r"$x^'$")
            
        if i == 1:
            xlab = "y"
            Ax[i][0].set_ylabel(r"$y^'$")

        for j in range(ncol):
            index = factor*j
            if factor*j == N_turns:
                index = -1
            X_hist_sub = X_hist[index].T

            Ax[i][j].plot(X_hist_sub[i*2], X_hist_sub[i*2 + 1], "o", mfc = 'black', ms = size)
            Ax[i][j].set_xlabel(xlab)

            Ax[i][j].set_xlim([-xbound_Max, xbound_Max])
            Ax[i][j].set_ylim([-ybound_Max, ybound_Max])
            Ax[0][j].set_title("%d turns"%(N_turns*j/(ncol - 1)))
            if j != 0:
                Ax[i][j].get_yaxis().set_visible(False)
    fig.savefig(save_txt)
    return Ax

# 1. Pulse/AC Skew Sextupole <a name="chapter1"></a>
## 1.1 Single Particle <a name="chapter1.1"></a>

In [7]:
# One particle
def Pulse(Twiss, N_turns, initial_turn, final_turn, j3):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y = Twiss
    M = M_phase(Twiss)
    
    x0 = np.sqrt(eps_x/gx)
    y0 = np.sqrt(eps_y/gy)
    xp0 = np.sqrt(eps_x/bx)
    yp0 = np.sqrt(eps_y/by)
    X_i = np.array([x0, xp0, y0, yp0])
    X_f = np.zeros(4)

    X_hist = np.zeros([N_turns, 4])
    X_hist[0] = X_i

    Eps_x = np.zeros(N_turns)
    Eps_y = np.zeros(N_turns)
    for i in range(N_turns):
        X_f = M @ X_i
        x, y = X_f[0], X_f[2]
        #Turn on Sextupole
        if i > initial_turn and i < final_turn:
            X_f[1] += j3*x*y
            X_f[3] += -j3/2*(y**2 - x**2)
        Eps_x[i] = epsilon_single(ax, bx, gx, x, X_f[1])
        Eps_y[i] = epsilon_single(ay, by, gy, y, X_f[3])
        X_hist[i] = X_f

        X_i = X_f

    X_hist = X_hist.T
    return X_hist, Eps_x, Eps_y

In [15]:
# Variables

#Twiss
eps_x = eps_y = 2.5e-6 #mrad
ax = ay = 0
bx = 8 #m
by = 2 #m
gx = (1 + ax**2)/bx
gy = (1 + ay**2)/by
nu_x = .2
nu_y = .4
Twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y

#Magnets
G_21L = 1
L = .75 #m https://www.bnl.gov/cad/accelerator/docs/pdf/rhicconfmanual.pdf p45
j3 = G_21L*(8*np.pi)/(np.sqrt(2)*bx*np.sqrt(by)*L) #m^-2
display(Math("j_3 = %f"%j3))

N_turns = 300
initial_turn = 100
final_turn = 240

X_hist, Eps_x, Eps_y = Pulse(Twiss, N_turns, initial_turn, final_turn, j3)
Animation_single(X_hist, 15, 50)

<IPython.core.display.Math object>

NameError: name 'Pulse' is not defined

In [9]:
Exchange_single(X_hist, "Single particle Pulsed Sextupole.png", 5, 3)

array([[<Axes: title={'center': '0 - 60 turns'}, xlabel='x', ylabel="$x^'$">,
        <Axes: title={'center': '60 - 120 turns'}, xlabel='x'>,
        <Axes: title={'center': '120 - 180 turns'}, xlabel='x'>,
        <Axes: title={'center': '180 - 240 turns'}, xlabel='x'>,
        <Axes: title={'center': '240 - 300 turns'}, xlabel='x'>],
       [<Axes: xlabel='y', ylabel="$y^'$">, <Axes: xlabel='y'>,
        <Axes: xlabel='y'>, <Axes: xlabel='y'>, <Axes: xlabel='y'>]],
      dtype=object)

In [10]:
plt.plot(Eps_x, label = r"$\epsilon_x$")
plt.plot(Eps_y, label = r"$\epsilon_y$")
plt.xlabel("# turns")
plt.ylabel(r"$\epsilon$")
plt.title('"Emittance" Exchange pulsed sextupole')
plt.axvline(x = initial_turn, color = "black")
plt.axvline(x = final_turn, color = "black")
plt.legend()
plt.show()

In [11]:
N_points = 300
G_int = np.linspace(0, 5, N_points + 1)
L = .75
J3 = G_int*(8*np.pi)/(np.sqrt(2)*bx*np.sqrt(by)*L)
print(J3[-1])
FEG = np.zeros(N_points + 1)

begin = time.time()
for i, j in enumerate(J3):
    X_hist, Eps_x, Eps_y = Pulse(Twiss, N_turns, initial_turn, final_turn, j)
    eps_x0 = Eps_x[0]
    eps_y0 = Eps_y[0]
    eps_xf = Eps_x[-1]
    eps_yf = Eps_y[-1]
    FEG[i] = np.abs((eps_x0 - eps_xf)/eps_x0) + np.abs((eps_y0 - eps_yf)/eps_y0)
end = time.time()
print("Time elapsed:", end - begin)


10.471975511965974
Time elapsed: 0.7441380023956299


In [13]:
plt.plot(G_int, FEG)
plt.title("Quality of Emittance Exchange")
plt.xlabel(r"$G_{int}$")
plt.ylabel("Fractional Emittance Growth")
plt.show()

## 1.2 Multiparticle <a name="chapter1.2"></a>

In [14]:
# Multiparticle
def Pulse2(Twiss, N_turns, N_part, initial_turn, final_turn, j3):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y = Twiss
    M = M_phase(Twiss)
    
    #Bounds of ellipse
    x0 = np.sqrt(eps_x/gx)
    y0 = np.sqrt(eps_y/gy)
    xp0 = np.sqrt(eps_x/bx)
    yp0 = np.sqrt(eps_y/by)
    Part_x = np.random.uniform(-x0, x0, N_part)
    Part_y = np.random.uniform(-y0, y0, N_part)
    Part_xp = np.random.uniform(-xp0, xp0, N_part)
    Part_yp = np.random.uniform(-yp0, yp0, N_part)
    X_i = []
    for i in range(N_part):
        if gx*Part_x[i]**2 + bx*Part_xp[i]**2 < eps_x and gy*Part_y[i]**2 + by*Part_yp[i]**2 < eps_x:
            X_i.append(np.array([Part_x[i], Part_xp[i], Part_y[i], Part_yp[i]]))

    N_part = len(X_i)
    X_i = np.array(X_i).T

    X_hist = np.zeros([N_turns, N_part, 4])
    X_f = np.zeros([N_part, 4])
    
    Eps_x = np.zeros(N_turns)
    Eps_y = np.zeros(N_turns)
    for i in range(N_turns):
        X_f = M @ X_i
        x, y = X_f[0], X_f[2]
        #Turn on Sextupole
        if i > initial_turn and i < final_turn:
            X_f[1] += j3*x*y
            X_f[3] += -j3/2*(y**2 - x**2)
        Eps_x[i] = epsilon_multi(X_f[0], X_f[1]) #epsilon(ax, bx, gx, np.max(np.abs(X_f.T[0])), np.max(np.abs(X_f.T[1])))
        Eps_y[i] = epsilon_multi(X_f[2], X_f[3]) #epsilon(ay, by, gy, np.max(np.abs(X_f.T[2])), np.max(np.abs(X_f.T[3]))) 
        X_hist[i] = X_f.T

        X_i = X_f

    return X_hist, Eps_x, Eps_y

In [71]:
# Variables

#Twiss
eps_x = eps_y = 2.5e-6 #m
ax = ay = 0
bx = 8 #m
by = 2 #m
gx = (1 + ax**2)/bx
gy = (1 + ay**2)/by
nu_x = .2
nu_y = .4
Twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y

#Magnets
G_21L = 1
L = .75 #m https://www.bnl.gov/cad/accelerator/docs/pdf/rhicconfmanual.pdf p45
j3 = G_21L*(8*np.pi)/(np.sqrt(2)*bx*np.sqrt(by)*L) #m^-2
display(Math("j_3 = %f"%j3))

N_turns = 300
N_part = 1000
initial_turn = 100
final_turn = 240
nrow = 2
ncol = 5

X_hist, Eps_x, Eps_y = Pulse2(Twiss, N_turns, int(N_part//.625), initial_turn, final_turn, j3)
Animation_multi(X_hist, 15, 50, nrow, ncol, 0)

<IPython.core.display.Math object>

In [16]:
Exchange_multi(X_hist, "Multiparticle Pulsed Sextupole.png", ncol = 5)

array([[<Axes: title={'center': '0 turns'}, xlabel='x', ylabel="$x^'$">,
        <Axes: title={'center': '75 turns'}, xlabel='x'>,
        <Axes: title={'center': '150 turns'}, xlabel='x'>,
        <Axes: title={'center': '225 turns'}, xlabel='x'>,
        <Axes: title={'center': '300 turns'}, xlabel='x'>],
       [<Axes: xlabel='y', ylabel="$y^'$">, <Axes: xlabel='y'>,
        <Axes: xlabel='y'>, <Axes: xlabel='y'>, <Axes: xlabel='y'>]],
      dtype=object)

In [17]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.plot(Eps_x, label = r"$\epsilon_x$")
ax.plot(Eps_y, label = r"$\epsilon_y$")

# kernel_size = 5
# kernel = np.ones(kernel_size) / kernel_size
# Eps_x_smooth = np.convolve(Eps_x, kernel, mode='valid')
# Eps_y_smooth = np.convolve(Eps_y, kernel, mode='valid')
# ax.plot(Eps_x_smooth, color = "blue", label = r"$\overline{\epsilon_x}$")
# ax.plot(Eps_y_smooth, color = "red", label = r"$\overline{\epsilon_y}$")
ax.set_xlabel("# turns")
ax.set_ylabel(r"$\epsilon$")
ax.set_title("Emittance Exchange pulsed sextupole")
ax.axvline(x = initial_turn, color = "black")
ax.axvline(x = final_turn, color = "black")
ax.legend()
fig.savefig("Emittance Exchange pulsed sextupole")
plt.show()
print("Ratio =", Eps_x[-1]/Eps_y[-1])

Ratio = 2.846035995528796


In [72]:
N_points = 300
G_int = np.linspace(0, 5, N_points + 1)
L = .75
J3 = G_int*(8*np.pi)/(np.sqrt(2)*bx*np.sqrt(by)*L)
print(J3[-1])
FEG = np.zeros(N_points + 1)

begin = time.time()
for i, j in enumerate(J3):
    X_hist, Eps_x, Eps_y = Pulse2(Twiss, N_turns, N_part, initial_turn, final_turn, j)
    eps_x0 = Eps_x[0]
    eps_y0 = Eps_y[0]
    eps_xf = Eps_x[-1]
    eps_yf = Eps_y[-1]
    FEG[i] = np.abs((eps_x0 - eps_xf)/eps_x0) + np.abs((eps_y0 - eps_yf)/eps_y0)
end = time.time()
print("Time elapsed:", end - begin)


10.471975511965974
Time elapsed: 26.23451018333435


In [73]:
fig, ax = plt.subplots(figsize = (10, 10))
# kernel_size = 20
# kernel = np.ones(kernel_size)/kernel_size
# FEG_smooth = np.convolve(FEG, kernel, mode = 'valid')
ax.plot(G_int, FEG)
ax.set_title("Quality of Emittance Exchange")
ax.set_xlabel(r"$G_{int}$")
ax.set_ylabel("Fractional Emittance Growth")
fig.savefig("Quality of Emittance Exchange for Pulsed Sextupole")
plt.show()

# 2. Resonance Crossing <a name="chapter2"></a>
## 2.1 Single Particle <a name="chapter2.1"></a>

In [12]:
# One particle crossing
def Cross(Twiss, N_turns, j3):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_xi, nu_xf, nu_y = Twiss
    Nu_x = np.linspace(nu_xi, nu_xf, N_turns + 1)
    
    x0 = np.sqrt(eps_x/gx)
    y0 = np.sqrt(eps_y/gy)
    xp0 = np.sqrt(eps_x/bx)
    yp0 = np.sqrt(eps_y/by)
    X_i = np.array([x0, xp0, y0, yp0])
    X_f = np.zeros(4)
    X_hist = np.zeros([N_turns, 4])
    X_hist[0] = X_i

    Eps_x = np.zeros(N_turns)
    Eps_y = np.zeros(N_turns)
    for i in range(N_turns):
        twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, Nu_x[i], nu_y
        M = M_phase(twiss)
        
        X_f = M @ X_i
        x, y = X_f[0], X_f[2]
        X_f[1] += j3*x*y
        X_f[3] += -j3/2*(y**2 - x**2)
        Eps_x[i] = epsilon_single(ax, bx, gx, x, X_f[1])
        Eps_y[i] = epsilon_single(ay, by, gy, y, X_f[3])
        X_hist[i] = X_f

        X_i = X_f

    X_hist = X_hist.T
    return X_hist, Eps_x, Eps_y, Nu_x

In [13]:
# Variables

#Twiss
eps_x = eps_y = 2.5e-6 #m
ax = ay = 0
bx = 8 #m
by = 2 #m
gx = (1 + ax**2)/bx
gy = (1 + ay**2)/by
nu_xi, nu_xf = .225, .175
#change nu_y
nu_y = .413
Twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_xi, nu_xf, nu_y

#Magnets
j3L = 4.7
L = .75
j3 = j3L/L
display(Math("j_3 = %f"%j3))
G_21L = np.sqrt(2)/(8*np.pi)*bx*np.sqrt(by)*j3L #m^-.5
display(Math("G_{2,-1,l} = %f"%G_21L))

N_turns = 5000
delta_dot = (nu_xf - nu_xi)/N_turns

X_hist, Eps_x, Eps_y, Nu_x = Cross(Twiss, N_turns, j3)
Animation_single(X_hist, 100, 1)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
Exchange_single(X_hist, "Single particle Resonance Crossing.png", 5, 1, Nu_x)

array([[<Axes: title={'center': '0 - 1000 turns'}, xlabel='x', ylabel="$x^'$">,
        <Axes: title={'center': '1000 - 2000 turns'}, xlabel='x'>,
        <Axes: title={'center': '2000 - 3000 turns'}, xlabel='x'>,
        <Axes: title={'center': '3000 - 4000 turns'}, xlabel='x'>,
        <Axes: title={'center': '4000 - 5000 turns'}, xlabel='x'>],
       [<Axes: xlabel='y', ylabel="$y^'$">, <Axes: xlabel='y'>,
        <Axes: xlabel='y'>, <Axes: xlabel='y'>, <Axes: xlabel='y'>]],
      dtype=object)

In [11]:
plt.plot(Eps_x, label = r"$\epsilon_x$")
plt.plot(Eps_y, label = r"$\epsilon_y$")

kernel_size = N_turns//20
kernel = np.ones(kernel_size) / kernel_size
Eps_x_smooth = np.convolve(Eps_x, kernel, mode='valid') #same, full
Eps_y_smooth = np.convolve(Eps_y, kernel, mode='valid')
plt.plot(Eps_x_smooth, color = "blue", label = r"$\overline{\epsilon_x}$")
plt.plot(Eps_y_smooth, color = "red", label = r"$\overline{\epsilon_y}$")
plt.xlabel("# turns")
plt.ylabel(r"$\epsilon$")
plt.title('"Emittance" Exchange Resonance Crossing')
plt.legend()
plt.show()

In [26]:
N_points = 100
J3 = np.linspace(0, 5, N_points + 1)
L = .75
G_21L = np.sqrt(2)/(8*np.pi)*bx*np.sqrt(by)*J3*L #m^-.5
G_eff = G_21L*np.sqrt(eps_x*np.pi/np.abs(delta_dot))
print(G_eff[-1])

spread = N_turns//10
FEG = np.zeros(N_points + 1)

begin = time.time()
for i, j in enumerate(J3):
    X_hist, Eps_x, Eps_y = Cross(Twiss, N_turns, j)
    eps_x0 = Eps_x[0]
    eps_y0 = Eps_y[0]
    eps_xf = np.mean(Eps_x[N_turns - spread: -1])
    eps_yf = np.mean(Eps_y[N_turns - spread: -1])
    FEG[i] = np.abs((eps_x0 - eps_xf)/eps_x0) + np.abs((eps_y0 - eps_yf)/eps_y0)
end = time.time()
print("Time elapsed:", end - begin)

2.1157109383040864
Time elapsed: 11.633929014205933


In [27]:
plt.plot(G_eff, FEG)
plt.title("Quality of Emittance Exchange")
plt.xlabel(r"$G_{eff}$")
plt.ylabel("Fractional Emittance Growth")
plt.show()

## 2.2 Multiparticle <a name="chapter2.2"></a>

In [17]:
# Multiparticle
def Cross2(Twiss, N_turns, N_part, j3):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_xi, nu_xf, nu_y = Twiss
    Nu_x = np.linspace(nu_xi, nu_xf, N_turns + 1)
    
    #Bounds of ellipse
    x0 = np.sqrt(eps_x/gx)
    y0 = np.sqrt(eps_y/gy)
    xp0 = np.sqrt(eps_x/bx)
    yp0 = np.sqrt(eps_y/by)
    Part_x = np.random.uniform(-x0, x0, N_part)
    Part_y = np.random.uniform(-y0, y0, N_part)
    Part_xp = np.random.uniform(-xp0, xp0, N_part)
    Part_yp = np.random.uniform(-yp0, yp0, N_part)
    X_i = []
    for i in range(N_part):
        if gx*Part_x[i]**2 + bx*Part_xp[i]**2 < eps_x and gy*Part_y[i]**2 + by*Part_yp[i]**2 < eps_x:
            X_i.append(np.array([Part_x[i], Part_xp[i], Part_y[i], Part_yp[i]]))

    N_part = len(X_i)
    X_i = np.array(X_i).T

    X_hist = np.zeros([N_turns, N_part, 4])
    X_hist[0] = X_i.T
    X_f = np.zeros([4, N_part])
    Eps_x = np.zeros(N_turns)
    Eps_y = np.zeros(N_turns)
    for i in range(N_turns):
        twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, Nu_x[i], nu_y
        M = M_phase(twiss)
        X_f = M @ X_i
        x, y = X_f[0], X_f[2]
        X_f[1] += j3*x*y
        X_f[3] += -j3/2*(y**2 - x**2)
        Eps_x[i] = epsilon_multi(X_f[0], X_f[1]) #epsilon(ax, bx, gx, np.max(np.abs(X_f.T[0])), np.max(np.abs(X_f.T[1])))
        Eps_y[i] = epsilon_multi(X_f[2], X_f[3]) #epsilon(ay, by, gy, np.max(np.abs(X_f.T[2])), np.max(np.abs(X_f.T[3]))) 
        X_hist[i] = X_f.T

        X_i = X_f

    return X_hist, Eps_x, Eps_y

In [61]:
# Variables

#Twiss
eps_x = eps_y = 2.5e-6 #m
ax = ay = 0
bx = 8 #m
by = 2 #m
gx = (1 + ax**2)/bx
gy = (1 + ay**2)/by
nu_xi, nu_xf = .225, .175
nu_y = .4
Twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_xi, nu_xf, nu_y

#Magnets
j3L = 4.7
L = .75
j3 = j3L/L
display(Math("j_3 = %f"%j3))
G_21L = np.sqrt(2)/(8*np.pi)*bx*np.sqrt(by)*j3L #m^-.5
display(Math("G_{2,-1,l} = %f"%G_21L))

N_turns = 5000
N_part = 1000
delta_dot = (nu_xf - nu_xi)/N_turns

begin = time.time()
X_hist, Eps_x, Eps_y = Cross2(Twiss, N_turns, int(N_part/.6), j3)
end = time.time()
print("Time elapsed:", end - begin)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Time elapsed: 2.304208993911743


In [63]:
Animation_multi(X_hist, 100, 1)

In [64]:
Exchange_multi(X_hist, "Multiparticle Resonance Crossing.png", 5, 1)

array([[<Axes: title={'center': '0 turns'}, xlabel='x', ylabel="$x^'$">,
        <Axes: title={'center': '1250 turns'}, xlabel='x'>,
        <Axes: title={'center': '2500 turns'}, xlabel='x'>,
        <Axes: title={'center': '3750 turns'}, xlabel='x'>,
        <Axes: title={'center': '5000 turns'}, xlabel='x'>],
       [<Axes: xlabel='y', ylabel="$y^'$">, <Axes: xlabel='y'>,
        <Axes: xlabel='y'>, <Axes: xlabel='y'>, <Axes: xlabel='y'>]],
      dtype=object)

In [66]:
fig, ax = plt.subplots(figsize = (10, 10))
plt.plot(Eps_x, label = r"$\epsilon_x$")
plt.plot(Eps_y, label = r"$\epsilon_y$")
# kernel_size = N_turns//20
# kernel = np.ones(kernel_size) / kernel_size
# Eps_x_smooth = np.convolve(Eps_x, kernel, mode='valid')
# Eps_y_smooth = np.convolve(Eps_y, kernel, mode='valid')
# plt.plot(Eps_x_smooth, color = "blue", label = r"$\overline{\epsilon_x}$")
# plt.plot(Eps_y_smooth, color = "red", label = r"$\overline{\epsilon_y}$")
ax.set_xlabel("# turns")
ax.set_ylabel(r"$\epsilon$")
ax.set_title("Emittance Exchange via Resonance crossing")
ax.legend()
fig.savefig("Emittance Exchange via Resonance crossing")
plt.show()
print("Ratio =", np.mean(Eps_x[int(N_turns*.8):])/np.mean(Eps_y[int(N_turns*.8):]))

Ratio = 4.187260221311241


In [69]:
N_points = 100
J3 = np.linspace(0, 15, N_points + 1)
L = .75
G_21L = np.sqrt(2)/(8*np.pi)*bx*np.sqrt(by)*J3*L #m^-.5
G_eff = G_21L*np.sqrt(eps_x*np.pi/np.abs(delta_dot))
print(G_eff[-1])
    
spread = N_turns//10
FEG = np.zeros(N_points + 1)

begin = time.time()
for i, j in enumerate(J3):
    X_hist, Eps_x, Eps_y = Cross2(Twiss, N_turns, N_part, j)
    eps_x0 = Eps_x[0]
    eps_y0 = Eps_y[0]
    eps_xf = np.mean(Eps_x[N_turns - spread: -1])
    eps_yf = np.mean(Eps_y[N_turns - spread: -1])
    FEG[i] = np.abs((eps_x0 - eps_xf)/eps_x0) + np.abs((eps_y0 - eps_yf)/eps_y0)
end = time.time()
print("Time elapsed:", end - begin)

6.34713281491226
Time elapsed: 211.91766595840454


In [70]:
fig, ax = plt.subplots(figsize = (10, 10))
plt.plot(G_eff, FEG)
ax.set_title("Quality of Emittance Exchange")
ax.set_xlabel(r"$G_{eff}$")
ax.set_ylabel("Fractional Emittance Growth")
fig.savefig("Quality of Emittance Exchange for Resonance Crossing")
plt.show()

Vary crossing speed

In [61]:
N_points = 100
N_Turns = np.linspace(1000, 5000, N_points + 1)
Delta_dot = (nu_xf - nu_xi)/N_Turns

spread = N_turns//10
FEG = np.zeros(N_points + 1)
begin = time.time()
for i, turns in enumerate(N_Turns):
    _, Eps_x, Eps_y = Cross2(Twiss, int(turns), int(N_part/.6), j3)
    eps_x0 = Eps_x[0]
    eps_y0 = Eps_y[0]
    eps_xf = np.mean(Eps_x[N_turns - spread: -1])
    eps_yf = np.mean(Eps_y[N_turns - spread: -1])
    FEG[i] = np.abs((eps_x0 - eps_xf)/eps_x0) + np.abs((eps_y0 - eps_yf)/eps_y0)
end = time.time()
print("Time elapsed:", end - begin)

Time elapsed: 104.19185304641724


In [67]:
fig, ax = plt.subplots(figsize = (10, 10))
plt.plot(Delta_dot, FEG)
ax.set_title("Quality of Emittance Exchange vs Crossing Speed")
ax.set_xlabel(r"$\dot{\delta}$")
ax.set_ylabel("Fractional Emittance Growth")
fig.savefig("Quality of Emittance Exchange vs Crossing Speed")
plt.show()

# 3. AC Skew Sextupole <a name="chapter3"></a>
## 3.1 Single Particle <a name="chapter3.1"></a>

In [34]:
# One particle crossing
def AC(Twiss, N_turns, j3):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y = Twiss
    M = M_phase(Twiss)
    nu_osc = abs(2*nu_x - nu_y)
    Nu_osc = np.linspace(0, nu_osc, N_turns + 1)
    t = np.linspace(0, N_turns, N_turns + 1)
    J3_osc = j3*np.sin(2*np.pi*Nu_osc*t)
#     plt.plot(t, J3_osc)
#     plt.show()
    
    x0 = np.sqrt(eps_x/gx)
    y0 = np.sqrt(eps_y/gy)
    xp0 = np.sqrt(eps_x/bx)
    yp0 = np.sqrt(eps_y/by)
    X_i = np.array([x0, xp0, y0, yp0])
    X_f = np.zeros(4)
    X_hist = np.zeros([N_turns, 4])
    X_hist[0] = X_i

    Eps_x = np.zeros(N_turns)
    Eps_y = np.zeros(N_turns)
    for i in range(N_turns):
        
        X_f = M @ X_i
        x, y = X_f[0], X_f[2]
        X_f[1] += J3_osc[i]*x*y
        X_f[3] += -J3_osc[i]/2*(y**2 - x**2)
        Eps_x[i] = epsilon_single(ax, bx, gx, x, X_f[1])
        Eps_y[i] = epsilon_single(ay, by, gy, y, X_f[3])
        X_hist[i] = X_f

        X_i = X_f

    X_hist = X_hist.T
    return X_hist, Eps_x, Eps_y

In [35]:
# Variables

#Twiss
eps_x = eps_y = 2.5e-6 #mrad
ax = ay = 0
bx = 8 #m
by = 2 #m
gx = (1 + ax**2)/bx
gy = (1 + ay**2)/by
nu_x = .2
nu_y = .42
Twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y

#Magnets
G_21L = 1
L = .75 #m https://www.bnl.gov/cad/accelerator/docs/pdf/rhicconfmanual.pdf p45
j3 = G_21L*(8*np.pi)/(np.sqrt(2)*bx*np.sqrt(by)*L) #m^-2
display(Math("j_3 = %f"%j3))

N_turns = 8000

X_hist, Eps_x, Eps_y = AC(Twiss, N_turns, j3)
Animation_single(X_hist, 50, 1)

<IPython.core.display.Math object>

In [36]:
Exchange_single(X_hist, "Single particle AC Sextupole.png", 5, 2)

array([[<Axes: title={'center': '0 - 1600 turns'}, xlabel='x', ylabel="$x^'$">,
        <Axes: title={'center': '1600 - 3200 turns'}, xlabel='x'>,
        <Axes: title={'center': '3200 - 4800 turns'}, xlabel='x'>,
        <Axes: title={'center': '4800 - 6400 turns'}, xlabel='x'>,
        <Axes: title={'center': '6400 - 8000 turns'}, xlabel='x'>],
       [<Axes: xlabel='y', ylabel="$y^'$">, <Axes: xlabel='y'>,
        <Axes: xlabel='y'>, <Axes: xlabel='y'>, <Axes: xlabel='y'>]],
      dtype=object)

In [37]:
plt.plot(Eps_x, label = r"$\epsilon_x$")
plt.plot(Eps_y, label = r"$\epsilon_y$")

kernel_size = 500
kernel = np.ones(kernel_size) / kernel_size
Eps_x_smooth = np.convolve(Eps_x, kernel, mode='valid')
Eps_y_smooth = np.convolve(Eps_y, kernel, mode='valid')
plt.plot(Eps_x_smooth, color = "blue", label = r"$\overline{\epsilon_x}$")
plt.plot(Eps_y_smooth, color = "red", label = r"$\overline{\epsilon_y}$")
plt.xlabel("# turns")
plt.ylabel(r"$\epsilon$")
plt.title('"Emittance" Exchange for AC sextupole')
plt.legend()
plt.show()

## 3.2 Multiparticle <a name="chapter3.2"></a>

In [38]:
# Multiparticle
def AC2(Twiss, N_turns, N_part, j3):
    ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y = Twiss
    M = M_phase(Twiss)
    nu_osc = abs(2*nu_x - nu_y)
    Nu_osc = np.linspace(0, nu_osc, N_turns + 1)
    t = np.linspace(0, N_turns, N_turns + 1)
    J3_osc = j3*np.sin(2*np.pi*Nu_osc*t)
    
    #Bounds of ellipse
    x0 = np.sqrt(eps_x/gx)
    y0 = np.sqrt(eps_y/gy)
    xp0 = np.sqrt(eps_x/bx)
    yp0 = np.sqrt(eps_y/by)
    Part_x = np.random.uniform(-x0, x0, N_part)
    Part_y = np.random.uniform(-y0, y0, N_part)
    Part_xp = np.random.uniform(-xp0, xp0, N_part)
    Part_yp = np.random.uniform(-yp0, yp0, N_part)
    X_i = []
    for i in range(N_part):
        if gx*Part_x[i]**2 + bx*Part_xp[i]**2 < eps_x and gy*Part_y[i]**2 + by*Part_yp[i]**2 < eps_x:
            X_i.append(np.array([Part_x[i], Part_xp[i], Part_y[i], Part_yp[i]]))

    N_part = len(X_i)
    X_i = np.array(X_i).T

    X_hist = np.zeros([N_turns, N_part, 4])
    X_hist[0] = X_i.T
    X_f = np.zeros([4, N_part])
    Eps_x = np.zeros(N_turns)
    Eps_y = np.zeros(N_turns)
    for i in range(N_turns):
        X_f = M @ X_i
        x, y = X_f[0], X_f[2]
        X_f[1] += J3_osc[i]*x*y
        X_f[3] += -J3_osc[i]/2*(y**2 - x**2)
        Eps_x[i] = epsilon_multi(X_f[0], X_f[1]) #epsilon(ax, bx, gx, np.max(np.abs(X_f.T[0])), np.max(np.abs(X_f.T[1])))
        Eps_y[i] = epsilon_multi(X_f[2], X_f[3]) #epsilon(ay, by, gy, np.max(np.abs(X_f.T[2])), np.max(np.abs(X_f.T[3]))) 
        X_hist[i] = X_f.T

        X_i = X_f

    return X_hist, Eps_x, Eps_y

In [74]:
# Variables

#Twiss
eps_x = eps_y = 2.5e-6 #m
ax = ay = 0
bx = 8 #m
by = 2 #m
gx = (1 + ax**2)/bx
gy = (1 + ay**2)/by
nu_x = .2
nu_y = .42
Twiss = ax, ay, bx, by, gx, gy, eps_x, eps_y, nu_x, nu_y

#Magnets
G_21L = 1
L = .75 #m https://www.bnl.gov/cad/accelerator/docs/pdf/rhicconfmanual.pdf p45
j3 = G_21L*(8*np.pi)/(np.sqrt(2)*bx*np.sqrt(by)*L) #m^-2
display(Math("j_3 = %f"%j3))

N_turns = 8000
N_part = 1000

begin = time.time()
X_hist, Eps_x, Eps_y = AC2(Twiss, N_turns, int(N_part/.6), j3)
end = time.time()
print("Time elapsed:", end - begin)

<IPython.core.display.Math object>

Time elapsed: 6.102197885513306


In [75]:
Animation_multi(X_hist, 50, .5)

In [41]:
Exchange_multi(X_hist, "Multiparticle AC Sextupole.png", 5, 1)

array([[<Axes: title={'center': '0 turns'}, xlabel='x', ylabel="$x^'$">,
        <Axes: title={'center': '2000 turns'}, xlabel='x'>,
        <Axes: title={'center': '4000 turns'}, xlabel='x'>,
        <Axes: title={'center': '6000 turns'}, xlabel='x'>,
        <Axes: title={'center': '8000 turns'}, xlabel='x'>],
       [<Axes: xlabel='y', ylabel="$y^'$">, <Axes: xlabel='y'>,
        <Axes: xlabel='y'>, <Axes: xlabel='y'>, <Axes: xlabel='y'>]],
      dtype=object)

In [42]:
fig, ax = plt.subplots(figsize = (10, 10))
plt.plot(Eps_x, label = r"$\epsilon_x$")
plt.plot(Eps_y, label = r"$\epsilon_y$")
# kernel_size = N_turns//50
# kernel = np.ones(kernel_size) / kernel_size
# Eps_x_smooth = np.convolve(Eps_x, kernel, mode='valid')
# Eps_y_smooth = np.convolve(Eps_y, kernel, mode='valid')
# plt.plot(Eps_x_smooth, color = "blue", label = r"$\overline{\epsilon_x}$")
# plt.plot(Eps_y_smooth, color = "red", label = r"$\overline{\epsilon_y}$")
ax.set_xlabel("# turns")
ax.set_ylabel(r"$\epsilon$")
ax.set_title("Emittance Exchange via AC Sextupole")
ax.legend()
fig.savefig("Emittance Exchange via AC Sextupole")
plt.show()
print("Ratio =", Eps_x[-1]/Eps_y[-1])


Ratio = 3.233183274465296


In [49]:
Ratio = np.linspace(.0001, 10000, 1000)
Lum = (1 + Ratio)/np.sqrt(Ratio)
plt.plot(Ratio, Lum)
plt.xscale('log')
plt.show()